In [ ]:
import os
print(os.getcwd())

In [ ]:
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from datetime import datetime, timedelta
import re
import csv
import os.path

In [ ]:
csv_filename = 'youtube_results.csv'
row_count = 0

with open(csv_filename, 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        row_count += 1

print(f"Total number of rows: {row_count}")

In [ ]:
import os
import re
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from datetime import datetime, timedelta

API_KEY = ''
youtube = build('youtube', 'v3', developerKey=API_KEY)
channel_ids = []
videos = []
try:
    for channel_id in channel_ids:
        print('Current channel: ', channel_id)
        uploads_id_response = youtube.channels().list(
            id=channel_id,
            part='contentDetails'
        ).execute()

        uploads_id = uploads_id_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

        next_page_token = None
        while True:
            playlistitems_list_request = youtube.playlistItems().list(
                playlistId=uploads_id,
                part='snippet',
                maxResults=50,
                pageToken=next_page_token
            )

            playlistitems_list_response = playlistitems_list_request.execute()

            for playlist_item in playlistitems_list_response['items']:
                video_id = playlist_item['snippet']['resourceId']['videoId']
                video_title = playlist_item['snippet']['title']
                    
                video_request = youtube.videos().list(
                    part='snippet,statistics,contentDetails,topicDetails',
                    id=video_id
                )    
                
                video_response = video_request.execute()
                
                if not video_response['items']:
                    continue

                video_details = video_response['items'][0]
                published_at = video_details['snippet']['publishedAt']
                channel_id = video_details['snippet']['channelId']
                description = video_details['snippet']['description']
                tags = video_details['snippet'].get('tags', [])
                category_id = video_details['snippet']['categoryId']
                duration = video_details['contentDetails']['duration']
                definition = video_details['contentDetails'].get('definition', '')
                caption = video_details['contentDetails'].get('caption', False)
                view_count = int(video_details['statistics'].get('viewCount', 0))
                like_count = video_details['statistics'].get('likeCount', 0)
                comment_count = video_details['statistics'].get('commentCount', 0)
                topic_categories = video_details.get('topicDetails', {}).get('topicCategories', [])
                duration_ms = video_details['contentDetails'].get('duration', '')

                channel_response = youtube.channels().list(
                    part='snippet,statistics,brandingSettings,topicDetails',
                    id=channel_id
                ).execute()

                channel_details = channel_response['items'][0]
                channel_title = channel_details['snippet']['title']
                channel_description = channel_details['snippet']['description']
                channel_keywords = channel_details['brandingSettings']['channel'].get('keywords', '')
                channel_subscriber_count = int(channel_details['statistics'].get('subscriberCount', 0))
                channel_video_count = channel_details['statistics'].get('videoCount', 0)
                channel_published_at = channel_details['snippet']['publishedAt']
                channel_topic_categories = channel_details.get('topicDetails', {}).get('topicCategories', [])

                videos.append({
                    'video_id': video_id,
                    'published_at': published_at,
                    'channel_id': channel_id,
                    'title': video_title,
                    'description': description,
                    'channel_title': channel_title,
                    'tags': tags,
                    'category_id': category_id,
                    'duration': duration,
                    'definition': definition,
                    'caption': caption,
                    'view_count': view_count,
                    'like_count': like_count,
                    'comment_count': comment_count,
                    'topic_categories': topic_categories,
                    'duration_ms': duration_ms,
                    'channel_description': channel_description,
                    'channel_keywords': channel_keywords,
                    'channel_subscriber_count': channel_subscriber_count,
                    'channel_video_count': channel_video_count,
                    'channel_published_at': channel_published_at, 
                    'channel_topic_categories': channel_topic_categories
                })

            #print('next page...')
            if 'nextPageToken' in playlistitems_list_response:
                next_page_token = playlistitems_list_response['nextPageToken']
            else:
                break

    print('exporting csv...')

    csv_filename = 'youtube_results.csv'
    if os.path.isfile(csv_filename):
        df = pd.read_csv(csv_filename)
    else:
        df = pd.DataFrame()

    new_df = pd.DataFrame(videos)
    df = pd.concat([df, new_df], ignore_index=True)

    df.drop_duplicates(subset='video_id', keep='last', inplace=True)
    df.to_csv(csv_filename, index=False)

    print(f"Results exported to '{csv_filename}'.")

except HttpError as e:
    csv_filename = 'youtube_results.csv'
    if os.path.isfile(csv_filename):
        df = pd.read_csv(csv_filename)
    else:
        df = pd.DataFrame()

    new_df = pd.DataFrame(videos)
    df = pd.concat([df, new_df], ignore_index=True)

    df.drop_duplicates(subset='video_id', keep='last', inplace=True)
    df.to_csv(csv_filename, index=False)

    print(f"Results exported to '{csv_filename}'.")
    
    if e.resp.status == 403 and 'quota' in str(e):
        print("Quota limit reached. Exporting data up to this point...")
    else:
        print(f"An error occurred: {str(e)}")
